In [1]:
import pandas as pd

In [2]:
data = pd.read_pickle('input_data.pickle')

print(data)

          result  odds_home  odds_draw  odds_away points_home points_away  \
index                                                                       
50.0    home win       1.20       6.00      15.00          15           3   
51.0    home win       1.13       8.00      19.00          15           6   
52.0        draw       2.10       3.30       3.50           6           4   
53.0    home win       1.67       3.75       5.00           5           3   
54.0    away win      10.00       5.50       1.29           3          13   
...          ...        ...        ...        ...         ...         ...   
3415.0      draw       1.44       4.33       7.50           8           2   
3416.0  away win       3.30       3.40       2.20           3           7   
3417.0  away win       2.10       4.00       3.00          10           7   
3418.0  home win       2.50       3.30       2.87           7          10   
3419.0  away win       2.62       3.30       2.70           7           8   

In [3]:
import numpy as np

features = data.drop(columns=["result", "odds_home", "odds_draw", "odds_away"])

X = np.array(features)

y = np.array(data["result"])
print(X[0])

[15 3 16 4 4 10 81 60 50 57 24 -1]


In [4]:
# 1. NAIVE BAYES

from sklearn.naive_bayes import GaussianNB

naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X, y)

GaussianNB()

In [5]:
# 2. NEURAL NETWORKS

from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
mlp_model.fit(X, y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [6]:
# 3. K-NN

from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=10)

knn_model.fit(X, y)

KNeighborsClassifier(n_neighbors=10)

In [9]:
# 4. BAYESIAN NETWORK

from pomegranate import *

features2 = data.drop(columns=["odds_home", "odds_draw", "odds_away", 
                              "shots_given_home", "shots_given_away", 
                              "shots_conceded_home", "shots_conceded_away", 
                              "corners_difference_home", "corners_difference_away"])

X_ = np.array(features2)

bayes_net_model = BayesianNetwork.from_samples(X_)

X_

array([['home win', 15, 3, ..., 4, 4, 10],
       ['home win', 15, 6, ..., 4, 4, 3],
       ['draw', 6, 4, ..., 3, 10, 6],
       ...,
       ['away win', 10, 7, ..., 10, 4, 6],
       ['home win', 7, 10, ..., 7, 5, 3],
       ['away win', 7, 8, ..., 4, 5, 3]], dtype=object)

In [8]:
print(bayes_net_model.predict([[None, 8, 7, 10, 4, 8, 4, 73, 52, 75, 68, 7, -12]])[0])

ValueError: Sample does not have the same number of dimensions as the model [None, 8, 7, 10, 4, 8, 4, 73, 52, 75, 68, 7, -12] 6